In [1]:
import networkx as nx
import itertools
import pandas as pd
import jieba
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np

In [2]:
def successive_list(alist):
    return [(alist[:-1][i], alist[1:][i]) for i in range(len(alist)-1)]

In [3]:
df_query = pd.DataFrame(columns = ['日期','排名','热搜词','热度'])
for filename in sorted(os.listdir('../爬虫/分小时数据')):
    path = '../爬虫/分小时数据/' + filename
    new = pd.read_excel(path)
    df_query = pd.concat([df_query,new],ignore_index=True)

In [4]:
# 修改日期格式
def date_format(dt):
    dt = str(dt)
    year = dt[:dt.find("年")]
    month = dt[dt.find("年")+1:dt.find("月")]
    day = dt[dt.find("月")+1:dt.find("日")]
    if len(month) == 1:
        month = '0' + month
    if len(day) == 1:
        day = '0' + day
    return year + '-' + month + '-' + day

# 新冠疫情

In [5]:
df = pd.read_excel("../相关性系数/相关系数排名-新冠疫情.xlsx")
query = list(df['关键词'][:500])
G=nx.Graph()

for words in df_query['热搜词']:
    slist = list(jieba.cut(words))
    if len(slist)>1:
        slist = [i  for i in slist if (len(i) > 1) and (i  in query)]
        edgelist = successive_list(slist)
#         G.add_edges_from(edgelist)
        for e1, e2 in edgelist:
            G.add_edge(e1, e2)
        


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lenovo\AppData\Local\Temp\jieba.cache
Loading model cost 1.905 seconds.
Prefix dict has been built successfully.


In [6]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-新冠疫情.xlsx")
df = df[list(df2['关键词'][:500])]
df1 = pd.read_excel("../相关性系数/新冠疫情确诊数据.xlsx",index_col=0)['新增确诊']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [7]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['新增确诊', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=146.4861, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=147.1069, p=0.0000  , df=1
likelihood ratio test: chi2=133.7080, p=0.0000  , df=1
parameter F test:         F=146.4861, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=82.4030 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=165.9749, p=0.0000  , df=2
likelihood ratio test: chi2=149.1514, p=0.0000  , df=2
parameter F test:         F=82.4030 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=146.4861, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=147.1069, p=0.0000  , df=1
likelihood ratio test: chi2=133.7080, p=0.0000  , df=1
parameter F test:         F=146.4861, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=82.4030 , p=0.0000  

ssr based F test:         F=79.4317 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=159.9902, p=0.0000  , df=2
likelihood ratio test: chi2=144.2840, p=0.0000  , df=2
parameter F test:         F=79.4317 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=144.4510, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=145.0631, p=0.0000  , df=1
likelihood ratio test: chi2=132.0126, p=0.0000  , df=1
parameter F test:         F=144.4510, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=79.4317 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=159.9902, p=0.0000  , df=2
likelihood ratio test: chi2=144.2840, p=0.0000  , df=2
parameter F test:         F=79.4317 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=144.4510, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test: 

ssr based chi2 test:   chi2=99.8875 , p=0.0000  , df=1
likelihood ratio test: chi2=93.4658 , p=0.0000  , df=1
parameter F test:         F=99.4660 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=57.9995 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=116.8218, p=0.0000  , df=2
likelihood ratio test: chi2=108.1502, p=0.0000  , df=2
parameter F test:         F=57.9995 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=99.4660 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=99.8875 , p=0.0000  , df=1
likelihood ratio test: chi2=93.4658 , p=0.0000  , df=1
parameter F test:         F=99.4660 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=57.9995 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=116.8218, p=0.0000  , df=2
likelihood ratio test: chi2=108.1502, p

parameter F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=186.3278, p=0.0000  , df=1
likelihood ratio test: chi2=165.4844, p=0.0000  , df=1
parameter F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=177.9147, p=0.0000  , df=2
likelihood ratio test: chi2=158.7636, p=0.0000  , df=2
parameter F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=186.3278, p=0.0000  , df=1
likelihood ratio test: chi2=165.4844, p=0.0000  , df=1
parameter F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1

Granger Causality
nu

parameter F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=177.9147, p=0.0000  , df=2
likelihood ratio test: chi2=158.7636, p=0.0000  , df=2
parameter F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=186.3278, p=0.0000  , df=1
likelihood ratio test: chi2=165.4844, p=0.0000  , df=1
parameter F test:         F=185.5416, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=177.9147, p=0.0000  , df=2
likelihood ratio test: chi2=158.7636, p=0.0000  , df=2
parameter F test:         F=88.3309 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
nu

...

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.5196  , p=0.0611  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=3.5345  , p=0.0601  , df=1
likelihood ratio test: chi2=3.5258  , p=0.0604  , df=1
parameter F test:         F=3.5196  , p=0.0611  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.6800  , p=0.1871  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=3.3839  , p=0.1842  , df=2
likelihood ratio test: chi2=3.3758  , p=0.1849  , df=2
parameter F test:         F=1.6800  , p=0.1871  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.2978 , p=0.0001  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=16.3669 , p=0.0001  , df=1
likelihood ratio test: chi2=16.1813 , p=0.0001  , df=1
parameter F test:         F=16.2978 , p=0.0001  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.8500  , p=0.0002  ,

ssr based F test:         F=8.0816  , p=0.0003  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=16.2778 , p=0.0003  , df=2
likelihood ratio test: chi2=16.0940 , p=0.0003  , df=2
parameter F test:         F=8.0816  , p=0.0003  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.2030  , p=0.2731  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=1.2081  , p=0.2717  , df=1
likelihood ratio test: chi2=1.2071  , p=0.2719  , df=1
parameter F test:         F=1.2030  , p=0.2731  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=8.0816  , p=0.0003  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=16.2778 , p=0.0003  , df=2
likelihood ratio test: chi2=16.0940 , p=0.0003  , df=2
parameter F test:         F=8.0816  , p=0.0003  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=448.4269, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test: 

number of lags (no zero) 2
ssr based F test:         F=32.7263 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=65.9169 , p=0.0000  , df=2
likelihood ratio test: chi2=63.0341 , p=0.0000  , df=2
parameter F test:         F=32.7263 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8929  , p=0.3450  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=0.8967  , p=0.3437  , df=1
likelihood ratio test: chi2=0.8961  , p=0.3438  , df=1
parameter F test:         F=0.8929  , p=0.3450  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=32.7263 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=65.9169 , p=0.0000  , df=2
likelihood ratio test: chi2=63.0341 , p=0.0000  , df=2
parameter F test:         F=32.7263 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.5212  , p=0.0109  , df_denom=708, df_

parameter F test:         F=12.7203 , p=0.0004  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.9011  , p=0.0029  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=11.8860 , p=0.0026  , df=2
likelihood ratio test: chi2=11.7876 , p=0.0028  , df=2
parameter F test:         F=5.9011  , p=0.0029  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.7203 , p=0.0004  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=12.7742 , p=0.0004  , df=1
likelihood ratio test: chi2=12.6608 , p=0.0004  , df=1
parameter F test:         F=12.7203 , p=0.0004  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.9011  , p=0.0029  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=11.8860 , p=0.0026  , df=2
likelihood ratio test: chi2=11.7876 , p=0.0028  , df=2
parameter F test:         F=5.9011  , p=0.0029  , df_denom=705, df_num=2

Granger Causality
nu

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=113.7570, p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=229.1275, p=0.0000  , df=2
likelihood ratio test: chi2=198.5773, p=0.0000  , df=2
parameter F test:         F=113.7570, p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=182.0794, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=182.8509, p=0.0000  , df=1
likelihood ratio test: chi2=162.7242, p=0.0000  , df=1
parameter F test:         F=182.0794, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=113.7570, p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=229.1275, p=0.0000  , df=2
likelihood ratio test: chi2=198.5773, p=0.0000  , df=2
parameter F test:         F=113.7570, p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=17.8487 , p=0.0000  ,

In [8]:
res_table = res_table.T
res_table = res_table[(res_table[0] ==1) &(res_table[1] ==1)]

In [9]:
res_table2 = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name, '新增确诊']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table2[column_name] = sig


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=54.9430 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=55.1758 , p=0.0000  , df=1
likelihood ratio test: chi2=53.1396 , p=0.0000  , df=1
parameter F test:         F=54.9430 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=41.0315 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=82.6450 , p=0.0000  , df=2
likelihood ratio test: chi2=78.1784 , p=0.0000  , df=2
parameter F test:         F=41.0315 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=54.9430 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=55.1758 , p=0.0000  , df=1
likelihood ratio test: chi2=53.1396 , p=0.0000  , df=1
parameter F test:         F=54.9430 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=41.0315 , p=0.0000  

ssr based F test:         F=30.2677 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=60.9648 , p=0.0000  , df=2
likelihood ratio test: chi2=58.4882 , p=0.0000  , df=2
parameter F test:         F=30.2677 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=57.0809 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=57.3228 , p=0.0000  , df=1
likelihood ratio test: chi2=55.1292 , p=0.0000  , df=1
parameter F test:         F=57.0809 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=30.2154 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=60.8593 , p=0.0000  , df=2
likelihood ratio test: chi2=58.3911 , p=0.0000  , df=2
parameter F test:         F=30.2154 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=57.0809 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test: 

parameter F test:         F=30.8745 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=61.5602 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=61.8210 , p=0.0000  , df=1
likelihood ratio test: chi2=59.2797 , p=0.0000  , df=1
parameter F test:         F=61.5602 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=29.9795 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=60.3842 , p=0.0000  , df=2
likelihood ratio test: chi2=57.9533 , p=0.0000  , df=2
parameter F test:         F=29.9795 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=61.5602 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=61.8210 , p=0.0000  , df=1
likelihood ratio test: chi2=59.2797 , p=0.0000  , df=1
parameter F test:         F=61.5602 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
nu

ssr based F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=28.7487 , p=0.0000  , df=2
likelihood ratio test: chi2=28.1819 , p=0.0000  , df=2
parameter F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.0809 , p=0.0001  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=16.1490 , p=0.0001  , df=1
likelihood ratio test: chi2=15.9683 , p=0.0001  , df=1
parameter F test:         F=16.0809 , p=0.0001  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=28.7487 , p=0.0000  , df=2
likelihood ratio test: chi2=28.1819 , p=0.0000  , df=2
parameter F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.0809 , p=0.0001  , df_denom=708, df_num=1
ssr based chi2 test: 

ssr based F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=28.7487 , p=0.0000  , df=2
likelihood ratio test: chi2=28.1819 , p=0.0000  , df=2
parameter F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.0809 , p=0.0001  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=16.1490 , p=0.0001  , df=1
likelihood ratio test: chi2=15.9683 , p=0.0001  , df=1
parameter F test:         F=16.0809 , p=0.0001  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=28.7487 , p=0.0000  , df=2
likelihood ratio test: chi2=28.1819 , p=0.0000  , df=2
parameter F test:         F=14.2731 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.0809 , p=0.0001  , df_denom=708, df_num=1
ssr based chi2 test: 

...

likelihood ratio test: chi2=17.4503 , p=0.0000  , df=1
parameter F test:         F=17.5917 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.0923 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=34.4271 , p=0.0000  , df=2
likelihood ratio test: chi2=33.6185 , p=0.0000  , df=2
parameter F test:         F=17.0923 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=17.5917 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=17.6662 , p=0.0000  , df=1
likelihood ratio test: chi2=17.4503 , p=0.0000  , df=1
parameter F test:         F=17.5917 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.0923 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=34.4271 , p=0.0000  , df=2
likelihood ratio test: chi2=33.6185 , p=0.0000  , df=2
parameter F test:         F=17.0923 , p

ssr based F test:         F=39.9590 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=40.1283 , p=0.0000  , df=1
likelihood ratio test: chi2=39.0368 , p=0.0000  , df=1
parameter F test:         F=39.9590 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=23.3370 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=47.0050 , p=0.0000  , df=2
likelihood ratio test: chi2=45.5145 , p=0.0000  , df=2
parameter F test:         F=23.3370 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=39.9590 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=40.1283 , p=0.0000  , df=1
likelihood ratio test: chi2=39.0368 , p=0.0000  , df=1
parameter F test:         F=39.9590 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=23.3370 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test: 

parameter F test:         F=8.9326  , p=0.0001  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=244.4375, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=245.4733, p=0.0000  , df=1
likelihood ratio test: chi2=210.8687, p=0.0000  , df=1
parameter F test:         F=244.4375, p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=155.0920, p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=312.3840, p=0.0000  , df=2
likelihood ratio test: chi2=258.8855, p=0.0000  , df=2
parameter F test:         F=155.0920, p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=244.4375, p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=245.4733, p=0.0000  , df=1
likelihood ratio test: chi2=210.8687, p=0.0000  , df=1
parameter F test:         F=244.4375, p=0.0000  , df_denom=708, df_num=1

Granger Causality
nu

ssr based F test:         F=15.7939 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=31.8117 , p=0.0000  , df=2
likelihood ratio test: chi2=31.1197 , p=0.0000  , df=2
parameter F test:         F=15.7939 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=20.8021 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=20.8902 , p=0.0000  , df=1
likelihood ratio test: chi2=20.5892 , p=0.0000  , df=1
parameter F test:         F=20.8021 , p=0.0000  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=18.1547 , p=0.0000  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=36.5669 , p=0.0000  , df=2
likelihood ratio test: chi2=35.6564 , p=0.0000  , df=2
parameter F test:         F=18.1547 , p=0.0000  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=20.8021 , p=0.0000  , df_denom=708, df_num=1
ssr based chi2 test: 

ssr based F test:         F=9.5275  , p=0.0001  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=19.1901 , p=0.0001  , df=2
likelihood ratio test: chi2=18.9353 , p=0.0001  , df=2
parameter F test:         F=9.5275  , p=0.0001  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.2037  , p=0.0739  , df_denom=708, df_num=1
ssr based chi2 test:   chi2=3.2173  , p=0.0729  , df=1
likelihood ratio test: chi2=3.2100  , p=0.0732  , df=1
parameter F test:         F=3.2037  , p=0.0739  , df_denom=708, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0308  , p=0.1320  , df_denom=705, df_num=2
ssr based chi2 test:   chi2=4.0903  , p=0.1294  , df=2
likelihood ratio test: chi2=4.0786  , p=0.1301  , df=2
parameter F test:         F=2.0308  , p=0.1320  , df_denom=705, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.2037  , p=0.0739  , df_denom=708, df_num=1
ssr based chi2 test: 

In [10]:
res_table2 = res_table2.T
res_table2 = res_table2[(res_table2[0] ==1) &(res_table2[1] ==1)]

In [11]:
results = dict()
results1 = dict()
results2 = dict()
results3 = dict()

In [12]:
result = []
for x in list(G.nodes):
    for y in list(res_table.index):
        if x == y:
            for z in list(res_table2.index):
                if x == z:
                    result.append(x)
                    break
            break

results['新冠疫情'] = result

In [33]:
# 网络 X 正向
result = []
for x in list(G.nodes):
    for y in list(res_table.index):
        if x == y:
                result.append(x)
                break

results1['新冠疫情'] = result

# 网络 X 反向
result = []
for x in list(G.nodes):
    for y in list(res_table2.index):
        if x == y:
                result.append(x)
                break

results2['新冠疫情'] = result

# 正向 X 反向
result = []
for x in list(res_table.index):
    for y in list(res_table2.index):
        if x == y:
                result.append(x)
                break


results3['新冠疫情'] = result

In [34]:
pd.DataFrame.from_dict(results1, orient='index').T.to_excel("选词-网络X正向-新冠疫情.xlsx",index=False)
pd.DataFrame.from_dict(results2, orient='index').T.to_excel("选词-网络X反向-新冠疫情.xlsx",index=False)
pd.DataFrame.from_dict(results3, orient='index').T.to_excel("选词-正向X反向-新冠疫情.xlsx",index=False)

In [13]:
pd.DataFrame.from_dict(results, orient='index').T.to_excel("选词-新冠疫情.xlsx",index=False)

# 中美汇率

In [15]:
df = pd.read_excel("../相关性系数/相关系数排名-中美汇率.xlsx")
query = list(df['关键词'][:100])
G=nx.Graph()

for words in df_query['热搜词']:
    slist = list(jieba.cut(words))
    if len(slist)>1:
        slist = [i  for i in slist if (len(i) > 1) and (i  in query)]
        edgelist = successive_list(slist)
#         G.add_edges_from(edgelist)
        for e1, e2 in edgelist:
            G.add_edge(e1, e2)

In [46]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-中美汇率.xlsx")
df = df[list(df2['关键词'][:100])]
df1 = pd.read_csv("../相关性系数/USD_CNY历史数据.csv")
df1['日期'] = df1['日期'].apply(date_format)
df1 = df1.sort_values(by='日期').set_index('日期')
df1 = df1['收盘']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [47]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['收盘', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2104  , p=0.6465  , df=1
likelihood ratio test: chi2=0.2104  , p=0.6465  , df=1
parameter F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1059  , p=0.8995  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.2139  , p=0.8986  , df=2
likelihood ratio test: chi2=0.2139  , p=0.8986  , df=2
parameter F test:         F=0.1059  , p=0.8995  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.2104  , p=0.6465  , df=1
likelihood ratio test: chi2=0.2104  , p=0.6465  , df=1
parameter F test:         F=0.2092  , p=0.6476  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1059  , p=0.8995  

ssr based F test:         F=1.1714  , p=0.3108  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=2.3662  , p=0.3063  , df=2
likelihood ratio test: chi2=2.3607  , p=0.3072  , df=2
parameter F test:         F=1.1714  , p=0.3108  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3610  , p=0.2439  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=1.3691  , p=0.2420  , df=1
likelihood ratio test: chi2=1.3673  , p=0.2423  , df=1
parameter F test:         F=1.3610  , p=0.2439  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5838  , p=0.5582  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.1792  , p=0.5545  , df=2
likelihood ratio test: chi2=1.1778  , p=0.5549  , df=2
parameter F test:         F=0.5838  , p=0.5582  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3610  , p=0.2439  , df_denom=503, df_num=1
ssr based chi2 test: 

parameter F test:         F=1.5215  , p=0.2180  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1544  , p=0.3161  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=2.3319  , p=0.3116  , df=2
likelihood ratio test: chi2=2.3265  , p=0.3125  , df=2
parameter F test:         F=1.1544  , p=0.3161  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1062  , p=0.7446  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1068  , p=0.7438  , df=1
likelihood ratio test: chi2=0.1068  , p=0.7438  , df=1
parameter F test:         F=0.1062  , p=0.7446  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7762  , p=0.4607  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.5679  , p=0.4566  , df=2
likelihood ratio test: chi2=1.5654  , p=0.4572  , df=2
parameter F test:         F=0.7762  , p=0.4607  , df_denom=500, df_num=2

Granger Causality
nu

likelihood ratio test: chi2=1.3937  , p=0.2378  , df=1
parameter F test:         F=1.3874  , p=0.2394  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5421  , p=0.5819  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.0949  , p=0.5784  , df=2
likelihood ratio test: chi2=1.0938  , p=0.5788  , df=2
parameter F test:         F=0.5421  , p=0.5819  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1537  , p=0.6952  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1546  , p=0.6942  , df=1
likelihood ratio test: chi2=0.1546  , p=0.6942  , df=1
parameter F test:         F=0.1537  , p=0.6952  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7946  , p=0.4523  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.6050  , p=0.4482  , df=2
likelihood ratio test: chi2=1.6025  , p=0.4488  , df=2
parameter F test:         F=0.7946  , p

ssr based chi2 test:   chi2=0.1302  , p=0.7182  , df=1
likelihood ratio test: chi2=0.1302  , p=0.7183  , df=1
parameter F test:         F=0.1294  , p=0.7192  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.7629  , p=0.0089  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=9.6210  , p=0.0081  , df=2
likelihood ratio test: chi2=9.5305  , p=0.0085  , df=2
parameter F test:         F=4.7629  , p=0.0089  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1294  , p=0.7192  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1302  , p=0.7182  , df=1
likelihood ratio test: chi2=0.1302  , p=0.7183  , df=1
parameter F test:         F=0.1294  , p=0.7192  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.7629  , p=0.0089  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=9.6210  , p=0.0081  , df=2
likelihood ratio test: chi2=9.5305  , p

...

ssr based F test:         F=13.9733 , p=0.0002  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=14.0566 , p=0.0002  , df=1
likelihood ratio test: chi2=13.8649 , p=0.0002  , df=1
parameter F test:         F=13.9733 , p=0.0002  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.2809  , p=0.0008  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=14.7074 , p=0.0006  , df=2
likelihood ratio test: chi2=14.4973 , p=0.0007  , df=2
parameter F test:         F=7.2809  , p=0.0008  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0590  , p=0.8082  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0593  , p=0.8076  , df=1
likelihood ratio test: chi2=0.0593  , p=0.8076  , df=1
parameter F test:         F=0.0590  , p=0.8082  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3039  , p=0.7381  , df_denom=500, df_num=2
ssr based chi2 test: 

parameter F test:         F=0.0115  , p=0.9146  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1167  , p=0.8899  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.2357  , p=0.8889  , df=2
likelihood ratio test: chi2=0.2356  , p=0.8889  , df=2
parameter F test:         F=0.1167  , p=0.8899  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0115  , p=0.9146  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0116  , p=0.9143  , df=1
likelihood ratio test: chi2=0.0116  , p=0.9143  , df=1
parameter F test:         F=0.0115  , p=0.9146  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1167  , p=0.8899  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.2357  , p=0.8889  , df=2
likelihood ratio test: chi2=0.2356  , p=0.8889  , df=2
parameter F test:         F=0.1167  , p=0.8899  , df_denom=500, df_num=2

Granger Causality
nu

number of lags (no zero) 1
ssr based F test:         F=0.0819  , p=0.7749  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0824  , p=0.7741  , df=1
likelihood ratio test: chi2=0.0824  , p=0.7741  , df=1
parameter F test:         F=0.0819  , p=0.7749  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4394  , p=0.6447  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.8876  , p=0.6416  , df=2
likelihood ratio test: chi2=0.8868  , p=0.6419  , df=2
parameter F test:         F=0.4394  , p=0.6447  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0819  , p=0.7749  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0824  , p=0.7741  , df=1
likelihood ratio test: chi2=0.0824  , p=0.7741  , df=1
parameter F test:         F=0.0819  , p=0.7749  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4394  , p=0.6447  , df_denom=500, df_

ssr based chi2 test:   chi2=0.0943  , p=0.7588  , df=1
likelihood ratio test: chi2=0.0943  , p=0.7588  , df=1
parameter F test:         F=0.0937  , p=0.7596  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6601  , p=0.5172  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.3334  , p=0.5134  , df=2
likelihood ratio test: chi2=1.3317  , p=0.5138  , df=2
parameter F test:         F=0.6601  , p=0.5172  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0937  , p=0.7596  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.0943  , p=0.7588  , df=1
likelihood ratio test: chi2=0.0943  , p=0.7588  , df=1
parameter F test:         F=0.0937  , p=0.7596  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6601  , p=0.5172  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=1.3334  , p=0.5134  , df=2
likelihood ratio test: chi2=1.3317  , p

likelihood ratio test: chi2=7.0973  , p=0.0288  , df=2
parameter F test:         F=3.5383  , p=0.0298  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1609  , p=0.6885  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1619  , p=0.6874  , df=1
likelihood ratio test: chi2=0.1619  , p=0.6874  , df=1
parameter F test:         F=0.1609  , p=0.6885  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1306  , p=0.8776  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=0.2638  , p=0.8764  , df=2
likelihood ratio test: chi2=0.2638  , p=0.8764  , df=2
parameter F test:         F=0.1306  , p=0.8776  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1609  , p=0.6885  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=0.1619  , p=0.6874  , df=1
likelihood ratio test: chi2=0.1619  , p=0.6874  , df=1
parameter F test:         F=0.1609  , p

In [48]:
res_table = res_table.T
res_table = res_table[(res_table[0] ==1) &(res_table[1] ==1)]

In [49]:
res_table2 = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name,'收盘']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table2[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=38.9122 , p=0.0000  , df=1
likelihood ratio test: chi2=37.4885 , p=0.0000  , df=1
parameter F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.8537 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=30.0045 , p=0.0000  , df=2
likelihood ratio test: chi2=29.1469 , p=0.0000  , df=2
parameter F test:         F=14.8537 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=38.9122 , p=0.0000  , df=1
likelihood ratio test: chi2=37.4885 , p=0.0000  , df=1
parameter F test:         F=38.6815 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=14.8537 , p=0.0000  

parameter F test:         F=13.9459 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=34.8911 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=35.0992 , p=0.0000  , df=1
likelihood ratio test: chi2=33.9354 , p=0.0000  , df=1
parameter F test:         F=34.8911 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=16.6156 , p=0.0000  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=33.5636 , p=0.0000  , df=2
likelihood ratio test: chi2=32.4953 , p=0.0000  , df=2
parameter F test:         F=16.6156 , p=0.0000  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=34.8911 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=35.0992 , p=0.0000  , df=1
likelihood ratio test: chi2=33.9354 , p=0.0000  , df=1
parameter F test:         F=34.8911 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
nu

likelihood ratio test: chi2=7.9790  , p=0.0185  , df=2
parameter F test:         F=3.9814  , p=0.0193  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=13.4070 , p=0.0003  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=13.4869 , p=0.0002  , df=1
likelihood ratio test: chi2=13.3103 , p=0.0003  , df=1
parameter F test:         F=13.4070 , p=0.0003  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.9814  , p=0.0193  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=8.0424  , p=0.0179  , df=2
likelihood ratio test: chi2=7.9790  , p=0.0185  , df=2
parameter F test:         F=3.9814  , p=0.0193  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=27.7143 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=27.8796 , p=0.0000  , df=1
likelihood ratio test: chi2=27.1386 , p=0.0000  , df=1
parameter F test:         F=27.7143 , p

ssr based F test:         F=11.2060 , p=0.0009  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=11.2728 , p=0.0008  , df=1
likelihood ratio test: chi2=11.1491 , p=0.0008  , df=1
parameter F test:         F=11.2060 , p=0.0009  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.5377  , p=0.0298  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=7.1462  , p=0.0281  , df=2
likelihood ratio test: chi2=7.0961  , p=0.0288  , df=2
parameter F test:         F=3.5377  , p=0.0298  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.2060 , p=0.0009  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=11.2728 , p=0.0008  , df=1
likelihood ratio test: chi2=11.1491 , p=0.0008  , df=1
parameter F test:         F=11.2060 , p=0.0009  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.5377  , p=0.0298  , df_denom=500, df_num=2
ssr based chi2 test: 

likelihood ratio test: chi2=18.3184 , p=0.0001  , df=2
parameter F test:         F=9.2350  , p=0.0001  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=21.9504 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=22.0813 , p=0.0000  , df=1
likelihood ratio test: chi2=21.6131 , p=0.0000  , df=1
parameter F test:         F=21.9504 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.3799  , p=0.0001  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=18.9474 , p=0.0001  , df=2
likelihood ratio test: chi2=18.6006 , p=0.0001  , df=2
parameter F test:         F=9.3799  , p=0.0001  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=21.9504 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=22.0813 , p=0.0000  , df=1
likelihood ratio test: chi2=21.6131 , p=0.0000  , df=1
parameter F test:         F=21.9504 , p

...

parameter F test:         F=4.5244  , p=0.0113  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=13.6669 , p=0.0002  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=13.7484 , p=0.0002  , df=1
likelihood ratio test: chi2=13.5650 , p=0.0002  , df=1
parameter F test:         F=13.6669 , p=0.0002  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.5244  , p=0.0113  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=9.1393  , p=0.0104  , df=2
likelihood ratio test: chi2=9.0576  , p=0.0108  , df=2
parameter F test:         F=4.5244  , p=0.0113  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=17.8851 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=17.9917 , p=0.0000  , df=1
likelihood ratio test: chi2=17.6793 , p=0.0000  , df=1
parameter F test:         F=17.8851 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
nu

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=9.6556  , p=0.0020  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=9.7132  , p=0.0018  , df=1
likelihood ratio test: chi2=9.6211  , p=0.0019  , df=1
parameter F test:         F=9.6556  , p=0.0020  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.1580  , p=0.0162  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=8.3992  , p=0.0150  , df=2
likelihood ratio test: chi2=8.3301  , p=0.0155  , df=2
parameter F test:         F=4.1580  , p=0.0162  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.2269 , p=0.0001  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=16.3237 , p=0.0001  , df=1
likelihood ratio test: chi2=16.0659 , p=0.0001  , df=1
parameter F test:         F=16.2269 , p=0.0001  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.1038  , p=0.0009  ,

ssr based F test:         F=7.3945  , p=0.0007  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=14.9369 , p=0.0006  , df=2
likelihood ratio test: chi2=14.7203 , p=0.0006  , df=2
parameter F test:         F=7.3945  , p=0.0007  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=18.7609 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=18.8728 , p=0.0000  , df=1
likelihood ratio test: chi2=18.5293 , p=0.0000  , df=1
parameter F test:         F=18.7609 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.3945  , p=0.0007  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=14.9369 , p=0.0006  , df=2
likelihood ratio test: chi2=14.7203 , p=0.0006  , df=2
parameter F test:         F=7.3945  , p=0.0007  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=13.0006 , p=0.0003  , df_denom=503, df_num=1
ssr based chi2 test: 

number of lags (no zero) 1
ssr based F test:         F=18.6569 , p=0.0000  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=18.7681 , p=0.0000  , df=1
likelihood ratio test: chi2=18.4284 , p=0.0000  , df=1
parameter F test:         F=18.6569 , p=0.0000  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.6867  , p=0.0036  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=11.4871 , p=0.0032  , df=2
likelihood ratio test: chi2=11.3584 , p=0.0034  , df=2
parameter F test:         F=5.6867  , p=0.0036  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.7363 , p=0.0004  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=12.8123 , p=0.0003  , df=1
likelihood ratio test: chi2=12.6528 , p=0.0004  , df=1
parameter F test:         F=12.7363 , p=0.0004  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.3189  , p=0.0001  , df_denom=500, df_

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.9003  , p=0.0089  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=6.9415  , p=0.0084  , df=1
likelihood ratio test: chi2=6.8943  , p=0.0086  , df=1
parameter F test:         F=6.9003  , p=0.0089  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.9640  , p=0.0525  , df_denom=500, df_num=2
ssr based chi2 test:   chi2=5.9873  , p=0.0501  , df=2
likelihood ratio test: chi2=5.9521  , p=0.0510  , df=2
parameter F test:         F=2.9640  , p=0.0525  , df_denom=500, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=6.9003  , p=0.0089  , df_denom=503, df_num=1
ssr based chi2 test:   chi2=6.9415  , p=0.0084  , df=1
likelihood ratio test: chi2=6.8943  , p=0.0086  , df=1
parameter F test:         F=6.9003  , p=0.0089  , df_denom=503, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.9640  , p=0.0525  ,

In [50]:
res_table2 = res_table2.T
res_table2 = res_table2[(res_table2[0] ==1) &(res_table2[1] ==1)]

In [51]:
result = []
for x in list(G.nodes):
    for y in list(res_table.index):
        if x == y:
            for z in list(res_table2.index):
                if x == z:
                    result.append(x)
                    break
            break

results['中美汇率'] = result

In [52]:
# 网络 X 正向
result = []
for x in list(G.nodes):
    for y in list(res_table.index):
        if x == y:
                result.append(x)
                break

results1['中美汇率'] = result

# 网络 X 反向
result = []
for x in list(G.nodes):
    for y in list(res_table2.index):
        if x == y:
                result.append(x)
                break

results2['中美汇率'] = result

# 正向 X 反向
result = []
for x in list(res_table.index):
    for y in list(res_table2.index):
        if x == y:
                result.append(x)
                break

results3['中美汇率'] = result

# 上证综指

In [17]:
df = pd.read_excel("../相关性系数/相关系数排名-中美汇率.xlsx")
query = list(df['关键词'][:100])
G=nx.Graph()

for words in df_query['热搜词']:
    slist = list(jieba.cut(words))
    if len(slist)>1:
        slist = [i  for i in slist if (len(i) > 1) and (i  in query)]
        edgelist = successive_list(slist)
#         G.add_edges_from(edgelist)
        for e1, e2 in edgelist:
            G.add_edge(e1, e2)

In [54]:
df = pd.read_excel("../相关性系数/日维度关键词热度排序总表.xlsx")
df = df.pivot_table(index='日期',columns='关键词',values='系数',aggfunc='sum').fillna(0)
df2 = pd.read_excel("../相关性系数/相关系数排名-上证综指.xlsx")
df = df[list(df2['关键词'][:100])]
df1 = pd.read_csv("../相关性系数/上证指数历史数据.csv")
df1['日期'] = df1['日期'].apply(date_format)
df1 = df1.sort_values(by='日期').set_index('日期')
df1 = df1['收盘']
df3 = pd.merge(df1, df, how='left',left_index=True,right_index=True)

In [55]:
res_table = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[['收盘', column_name]], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table[column_name] = sig



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=12.3281 , p=0.0004  , df=1
likelihood ratio test: chi2=12.1741 , p=0.0005  , df=1
parameter F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4973  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=19.1928 , p=0.0001  , df=2
likelihood ratio test: chi2=18.8221 , p=0.0001  , df=2
parameter F test:         F=9.4973  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=12.3281 , p=0.0004  , df=1
likelihood ratio test: chi2=12.1741 , p=0.0005  , df=1
parameter F test:         F=12.2519 , p=0.0005  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.4973  , p=0.0001  

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.9679  , p=0.3257  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.9739  , p=0.3237  , df=1
likelihood ratio test: chi2=0.9729  , p=0.3240  , df=1
parameter F test:         F=0.9679  , p=0.3257  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0423  , p=0.1309  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=4.1272  , p=0.1270  , df=2
likelihood ratio test: chi2=4.1097  , p=0.1281  , df=2
parameter F test:         F=2.0423  , p=0.1309  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.9679  , p=0.3257  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.9739  , p=0.3237  , df=1
likelihood ratio test: chi2=0.9729  , p=0.3240  , df=1
parameter F test:         F=0.9679  , p=0.3257  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0423  , p=0.1309  ,

ssr based chi2 test:   chi2=0.3043  , p=0.5812  , df=1
likelihood ratio test: chi2=0.3043  , p=0.5812  , df=1
parameter F test:         F=0.3025  , p=0.5826  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.2463  , p=0.7818  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.4978  , p=0.7797  , df=2
likelihood ratio test: chi2=0.4976  , p=0.7798  , df=2
parameter F test:         F=0.2463  , p=0.7818  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.2917  , p=0.0072  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=7.3371  , p=0.0068  , df=1
likelihood ratio test: chi2=7.2822  , p=0.0070  , df=1
parameter F test:         F=7.2917  , p=0.0072  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=6.2696  , p=0.0021  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=12.6700 , p=0.0018  , df=2
likelihood ratio test: chi2=12.5070 , p

ssr based F test:         F=0.1132  , p=0.7367  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1139  , p=0.7358  , df=1
likelihood ratio test: chi2=0.1139  , p=0.7358  , df=1
parameter F test:         F=0.1132  , p=0.7367  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0584  , p=0.9433  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.1180  , p=0.9427  , df=2
likelihood ratio test: chi2=0.1180  , p=0.9427  , df=2
parameter F test:         F=0.0584  , p=0.9433  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.5220  , p=0.0192  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=5.5563  , p=0.0184  , df=1
likelihood ratio test: chi2=5.5248  , p=0.0187  , df=1
parameter F test:         F=5.5220  , p=0.0192  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.5005  , p=0.0116  , df_denom=479, df_num=2
ssr based chi2 test: 

number of lags (no zero) 1
ssr based F test:         F=0.0121  , p=0.9125  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0122  , p=0.9122  , df=1
likelihood ratio test: chi2=0.0122  , p=0.9122  , df=1
parameter F test:         F=0.0121  , p=0.9125  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.3840  , p=0.2516  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=2.7968  , p=0.2470  , df=2
likelihood ratio test: chi2=2.7888  , p=0.2480  , df=2
parameter F test:         F=1.3840  , p=0.2516  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0121  , p=0.9125  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.0122  , p=0.9122  , df=1
likelihood ratio test: chi2=0.0122  , p=0.9122  , df=1
parameter F test:         F=0.0121  , p=0.9125  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.3840  , p=0.2516  , df_denom=479, df_

...

ssr based chi2 test:   chi2=1.3576  , p=0.2439  , df=1
likelihood ratio test: chi2=1.3557  , p=0.2443  , df=1
parameter F test:         F=1.3492  , p=0.2460  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8639  , p=0.4222  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.7459  , p=0.4177  , df=2
likelihood ratio test: chi2=1.7427  , p=0.4184  , df=2
parameter F test:         F=0.8639  , p=0.4222  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3492  , p=0.2460  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.3576  , p=0.2439  , df=1
likelihood ratio test: chi2=1.3557  , p=0.2443  , df=1
parameter F test:         F=1.3492  , p=0.2460  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8639  , p=0.4222  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.7459  , p=0.4177  , df=2
likelihood ratio test: chi2=1.7427  , p

parameter F test:         F=1.4932  , p=0.2223  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7584  , p=0.4690  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.5327  , p=0.4647  , df=2
likelihood ratio test: chi2=1.5303  , p=0.4653  , df=2
parameter F test:         F=0.7584  , p=0.4690  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4932  , p=0.2223  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=1.5024  , p=0.2203  , df=1
likelihood ratio test: chi2=1.5001  , p=0.2207  , df=1
parameter F test:         F=1.4932  , p=0.2223  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7584  , p=0.4690  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.5327  , p=0.4647  , df=2
likelihood ratio test: chi2=1.5303  , p=0.4653  , df=2
parameter F test:         F=0.7584  , p=0.4690  , df_denom=479, df_num=2

Granger Causality
nu

number of lags (no zero) 1
ssr based F test:         F=0.1203  , p=0.7289  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1210  , p=0.7279  , df=1
likelihood ratio test: chi2=0.1210  , p=0.7279  , df=1
parameter F test:         F=0.1203  , p=0.7289  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1587  , p=0.8533  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=0.3207  , p=0.8518  , df=2
likelihood ratio test: chi2=0.3206  , p=0.8519  , df=2
parameter F test:         F=0.1587  , p=0.8533  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1203  , p=0.7289  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=0.1210  , p=0.7279  , df=1
likelihood ratio test: chi2=0.1210  , p=0.7279  , df=1
parameter F test:         F=0.1203  , p=0.7289  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1587  , p=0.8533  , df_denom=479, df_

ssr based F test:         F=2.8903  , p=0.0565  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=5.8409  , p=0.0539  , df=2
likelihood ratio test: chi2=5.8059  , p=0.0549  , df=2
parameter F test:         F=2.8903  , p=0.0565  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.9747  , p=0.0852  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=2.9933  , p=0.0836  , df=1
likelihood ratio test: chi2=2.9841  , p=0.0841  , df=1
parameter F test:         F=2.9747  , p=0.0852  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.8903  , p=0.0565  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=5.8409  , p=0.0539  , df=2
likelihood ratio test: chi2=5.8059  , p=0.0549  , df=2
parameter F test:         F=2.8903  , p=0.0565  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=17.3893 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test: 

ssr based chi2 test:   chi2=0.0060  , p=0.9382  , df=1
likelihood ratio test: chi2=0.0060  , p=0.9382  , df=1
parameter F test:         F=0.0060  , p=0.9384  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7549  , p=0.4706  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=1.5256  , p=0.4664  , df=2
likelihood ratio test: chi2=1.5232  , p=0.4669  , df=2
parameter F test:         F=0.7549  , p=0.4706  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=11.4243 , p=0.0008  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=11.4954 , p=0.0007  , df=1
likelihood ratio test: chi2=11.3613 , p=0.0007  , df=1
parameter F test:         F=11.4243 , p=0.0008  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.7297 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=25.7251 , p=0.0000  , df=2
likelihood ratio test: chi2=25.0648 , p

In [56]:
res_table = res_table.T
res_table = res_table[(res_table[0] ==1) &(res_table[1] ==1)]

In [57]:
res_table2 = pd.DataFrame()
maxlag = 2  # lag值
for column_name in df3.columns[1:]:
    sig = []
    for i in range(maxlag):
        pvalue = grangercausalitytests(df3[[column_name,'收盘']], maxlag=maxlag)[i+1][0][ 'params_ftest'][1]
        if pvalue < 0.05:
            sig.append(1)
        else:
            sig.append(0)
        
    res_table2[column_name] = sig


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=46.6679 , p=0.0000  , df=1
likelihood ratio test: chi2=44.5570 , p=0.0000  , df=1
parameter F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.7455 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=35.8614 , p=0.0000  , df=2
likelihood ratio test: chi2=34.5951 , p=0.0000  , df=2
parameter F test:         F=17.7455 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=46.6679 , p=0.0000  , df=1
likelihood ratio test: chi2=44.5570 , p=0.0000  , df=1
parameter F test:         F=46.3792 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=17.7455 , p=0.0000  


Granger Causality
number of lags (no zero) 2
ssr based F test:         F=7.5773  , p=0.0006  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=15.3128 , p=0.0005  , df=2
likelihood ratio test: chi2=15.0756 , p=0.0005  , df=2
parameter F test:         F=7.5773  , p=0.0006  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=39.9298 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=40.1784 , p=0.0000  , df=1
likelihood ratio test: chi2=38.6007 , p=0.0000  , df=1
parameter F test:         F=39.9298 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=19.6697 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=39.7500 , p=0.0000  , df=2
likelihood ratio test: chi2=38.2019 , p=0.0000  , df=2
parameter F test:         F=19.6697 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=39.9298 , p=0.0000  

ssr based F test:         F=35.7415 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=35.9639 , p=0.0000  , df=1
likelihood ratio test: chi2=34.6930 , p=0.0000  , df=1
parameter F test:         F=35.7415 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=10.9423 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=22.1131 , p=0.0000  , df=2
likelihood ratio test: chi2=21.6228 , p=0.0000  , df=2
parameter F test:         F=10.9423 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=35.7415 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=35.9639 , p=0.0000  , df=1
likelihood ratio test: chi2=34.6930 , p=0.0000  , df=1
parameter F test:         F=35.7415 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=10.9423 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test: 

ssr based F test:         F=9.3868  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=18.9696 , p=0.0001  , df=2
likelihood ratio test: chi2=18.6073 , p=0.0001  , df=2
parameter F test:         F=9.3868  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=19.7056 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=19.8282 , p=0.0000  , df=1
likelihood ratio test: chi2=19.4336 , p=0.0000  , df=1
parameter F test:         F=19.7056 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.3868  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=18.9696 , p=0.0001  , df=2
likelihood ratio test: chi2=18.6073 , p=0.0001  , df=2
parameter F test:         F=9.3868  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=16.3871 , p=0.0001  , df_denom=482, df_num=1
ssr based chi2 test: 

ssr based chi2 test:   chi2=9.4172  , p=0.0090  , df=2
likelihood ratio test: chi2=9.3267  , p=0.0094  , df=2
parameter F test:         F=4.6599  , p=0.0099  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=21.7624 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=21.8979 , p=0.0000  , df=1
likelihood ratio test: chi2=21.4179 , p=0.0000  , df=1
parameter F test:         F=21.7624 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=9.5218  , p=0.0001  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=19.2424 , p=0.0001  , df=2
likelihood ratio test: chi2=18.8698 , p=0.0001  , df=2
parameter F test:         F=9.5218  , p=0.0001  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=21.7624 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=21.8979 , p=0.0000  , df=1
likelihood ratio test: chi2=21.4179 , p

...

likelihood ratio test: chi2=15.7716 , p=0.0001  , df=1
parameter F test:         F=15.9317 , p=0.0001  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=6.2246  , p=0.0021  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=12.5791 , p=0.0019  , df=2
likelihood ratio test: chi2=12.4184 , p=0.0020  , df=2
parameter F test:         F=6.2246  , p=0.0021  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.8817 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=29.0615 , p=0.0000  , df=1
likelihood ratio test: chi2=28.2241 , p=0.0000  , df=1
parameter F test:         F=28.8817 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=12.0087 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=24.2680 , p=0.0000  , df=2
likelihood ratio test: chi2=23.6792 , p=0.0000  , df=2
parameter F test:         F=12.0087 , p

ssr based chi2 test:   chi2=10.6538 , p=0.0011  , df=1
likelihood ratio test: chi2=10.5385 , p=0.0012  , df=1
parameter F test:         F=10.5879 , p=0.0012  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.1307  , p=0.0062  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=10.3686 , p=0.0056  , df=2
likelihood ratio test: chi2=10.2591 , p=0.0059  , df=2
parameter F test:         F=5.1307  , p=0.0062  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=10.5879 , p=0.0012  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=10.6538 , p=0.0011  , df=1
likelihood ratio test: chi2=10.5385 , p=0.0012  , df=1
parameter F test:         F=10.5879 , p=0.0012  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.1307  , p=0.0062  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=10.3686 , p=0.0056  , df=2
likelihood ratio test: chi2=10.2591 , p

ssr based F test:         F=10.2053 , p=0.0000  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=20.6237 , p=0.0000  , df=2
likelihood ratio test: chi2=20.1964 , p=0.0000  , df=2
parameter F test:         F=10.2053 , p=0.0000  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.7540  , p=0.0533  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=3.7773  , p=0.0520  , df=1
likelihood ratio test: chi2=3.7627  , p=0.0524  , df=1
parameter F test:         F=3.7540  , p=0.0533  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.8088  , p=0.0228  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=7.6972  , p=0.0213  , df=2
likelihood ratio test: chi2=7.6366  , p=0.0220  , df=2
parameter F test:         F=3.8088  , p=0.0228  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.7540  , p=0.0533  , df_denom=482, df_num=1
ssr based chi2 test: 

ssr based F test:         F=3.9154  , p=0.0206  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=7.9125  , p=0.0191  , df=2
likelihood ratio test: chi2=7.8486  , p=0.0198  , df=2
parameter F test:         F=3.9154  , p=0.0206  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=18.6384 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=18.7544 , p=0.0000  , df=1
likelihood ratio test: chi2=18.4009 , p=0.0000  , df=1
parameter F test:         F=18.6384 , p=0.0000  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.1019  , p=0.0064  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=10.3103 , p=0.0058  , df=2
likelihood ratio test: chi2=10.2020 , p=0.0061  , df=2
parameter F test:         F=5.1019  , p=0.0064  , df_denom=479, df_num=2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=18.6384 , p=0.0000  , df_denom=482, df_num=1
ssr based chi2 test: 

number of lags (no zero) 1
ssr based F test:         F=11.3508 , p=0.0008  , df_denom=482, df_num=1
ssr based chi2 test:   chi2=11.4215 , p=0.0007  , df=1
likelihood ratio test: chi2=11.2891 , p=0.0008  , df=1
parameter F test:         F=11.3508 , p=0.0008  , df_denom=482, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=5.2030  , p=0.0058  , df_denom=479, df_num=2
ssr based chi2 test:   chi2=10.5146 , p=0.0052  , df=2
likelihood ratio test: chi2=10.4020 , p=0.0055  , df=2
parameter F test:         F=5.2030  , p=0.0058  , df_denom=479, df_num=2


In [58]:
res_table2 = res_table2.T
res_table2 = res_table2[(res_table2[0] ==1) &(res_table2[1] ==1)]

In [59]:
result = []
for x in list(G.nodes):
    for y in list(res_table.index):
        if x == y:
            for z in list(res_table2.index):
                if x == z:
                    result.append(x)
                    break
            break

results['上证综指'] = result

In [60]:
# 网络 X 正向
result = []
for x in list(G.nodes):
    for y in list(res_table.index):
        if x == y:
                result.append(x)
                break

results1['上证综指'] = result

# 网络 X 反向
result = []
for x in list(G.nodes):
    for y in list(res_table2.index):
        if x == y:
                result.append(x)
                break

results2['上证综指'] = result

# 正向 X 反向
result = []
for x in list(res_table.index):
    for y in list(res_table2.index):
        if x == y:
                result.append(x)
                break


results3['上证综指'] = result

# 输出

In [87]:
pd.DataFrame.from_dict(results, orient='index').T.to_excel("选词.xlsx",index=False)

In [62]:
pd.DataFrame.from_dict(results1, orient='index').T.to_excel("选词-网络X正向.xlsx",index=False)
pd.DataFrame.from_dict(results2, orient='index').T.to_excel("选词-网络X反向.xlsx",index=False)
pd.DataFrame.from_dict(results3, orient='index').T.to_excel("选词-正向X反向.xlsx",index=False)